# 凸优化的收敛性质
<font color=blue>注1：这里不考虑深度网络，因为深度网络一般都是非凸函数</font> \
<font color=blue>注2：这部分也没有考虑理论目标和训练目标之间的gap，详见非凸收敛分析</font>

## I. L(w) is quadratic, 泰勒精确展开，可以一步求解
### I.1 一维，一元二次方程
1. **形态**：$f(w)=f(w_{0})+f^{'}(w_{0})(w-w_{0})+\frac{1}{2}f^{''}(w_{0})(w-w_{0})^{2}$
2. **solution**：$w_{min}=w_{0}-\frac{1}{f^{''}(w_{0})}*f^{'}(w_{0})$\
相当于GD中：$η_{opt}=\frac{1}{f^{''}(w_{0})}$，取$η=η_{opt}$\
此时有：
$$
\begin{align}
& η > 2*η_{opt}，diverge\\
& η < η_{opt}， converge\\
& η_{opt} < η < 2*η_{opt}，oscillate
\end{align}
$$
3. **convergence rate**：一步求解

### I.2 多维，多元二次方程：$f(W)=\frac{1}{2}W^{T}AW+W^{T}B + C$
<font color=green>此时$Hessian=\frac{\partial^2 f}{\partial w_i\partial w_j}=A，condition\ number=\frac{max\lambda_i}{min\lambda_i}$,$\lambda$是Hessian Metrix的特征值。</font>
#### I.2.1 如果A是diagonal，此时$w_{i}$相互独立
1. GD solution: \
$w^{[t+1]} = w^{[t]} -\eta*\nabla _{w} f(w)$ \
$\eta_{i, opt}=\frac{1}{a_{ii}}$，$a_{ii}$是A的eigen values，也是A对角线上的第i个元素 \
2. $\eta$的不同形式: \
(1) 如果$\eta$是scalar，各个维度用同一个learning rate，则：$\eta<2*\underset{i}{min}\ \eta_{i, opt}$。此时需要迭代，如果condition number太大，收敛会很慢 \
(2) 如果$\eta$为vector，$\eta=A^{-1}$, 不同的维度用不同的learning rate。收敛一步到位

#### I.2.2 如果A不是diagonal：对A做矩阵分解，转diagonal
1. 迭代分析: 
$$\begin{align} 
f(W)& = f(W_{min})+\frac{1}{2}(W-W_{min})^{T}H(W_{min})(W-W_{min})   ……[1]\\
& =f(W_{min})+\frac{1}{2}(W-W_{min})^{T}A(W-W_{min}) \\
迭代式：\\
w^{[t+1]} & = w^{[t]} -η*\nabla _{w} f(w) \\
& =w^{[t]}-η*H(W_{min})(W-W_{min})\\
& =w^{[t]}-η*A*(W-W_{min})\\
(w^{[t+1]}-W_{min})&=(I-η*A)(w^{[t]}-W_{min})
\end{align}
$$

2. 数学处理：change coordinates,取$Λ=ΘAΘ^{T}$，$ν=Θ(W-W_{min})$上面[1]式转化为diagonal form：
$$f(ν)\approx f(0)+\frac{1}{2}ν^{T}Λv$$
对应的迭代方式转换为：
$$ν^{[t+1]}=(I-ηΛ)*ν^{[t]}$$
solution：取$\eta$为vector，$\eta=Λ^{-1}$, 不同的维度用不同的learning rate。代入有：
$$ν^{[t+1]}=(I-ηΛ)*ν^{[t]}=0$$
因此也可以做到一步收敛到位。
3. 问题：<font color=red>但这种方法要对Hessian做分解，复杂度是$O(n^{3})$</font>

## II. L(w) is generic convex function, 泰勒近似展开，Newton’s method
### II.1 一维： 
$$f(w)\approx f(w_{0})+f^{'}(w_{0})(w-w_{0})+\frac{1}{2}f^{''}(w_{0})(w-w_{0})^{2}$$
1. **solution：Newton’s method，跟GD的区别在于，learning rate用Hessian**
$$w^{[t+1]} = w^{[t]} -\eta*\nabla_{w} f(w)$$
(1) 记$\eta_{opt}=\frac{1}{f^{''}(w_{0})}$，但此时$\eta_{opt}$并不是真正的“最优learning rate”，只是近似 \
(2) 取$\eta=\eta_{opt}$进行迭代
2. **convergence rate**：**linearly**

### II.2 多维：
$$\begin{align}
f(W) \
& \approx f(W_{0})+\bigtriangledown_{w}f(W_{0})(W-W_{0})+\frac{1}{2}(W-W_{0})^{T}H(W-W_{0}\\
& \approx f(W_{min})+\frac{1}{2}(W-W_{min})^{T}H(W_{min})(W-W_{min})……[1]\\
迭代式： \\
w^{[t+1]} & = w^{[t]} -η*\bigtriangledown _{w} f(w) \\
& \approx w^{[t]}-η*H(W_{min})(W-W_{min}) \\
(w^{[t+1]}-W_{min})& \approx(I-η*H(W_{min}))(w^{[t]}-W_{min})
\end{align}$$   

#### II.2.1 solution
1. **Newton's method：**
$$取\eta=H^{-1}(W_{min}), 有(w^{[t+1]}-W_{min})\approx0$$
如果限制各个维度learning rate是统一的，用  $\eta_{opt}=\frac{1}{\underset{i}{max}{λ_{i}}}$
2. Batch GD \
converge的条件：$(I-\eta*H(W_{min}))$ always shrinks vector，也就是在任意维度i上满足：$|1-\eta*λ_{i}|<1$ ⇒ $η<\frac{2}{\underset{i}{max}{λ_{i}}}$

#### II.2.2 change coordinates: *[注，不是whiten]* 
取$Λ=ΘHΘ^{T}$，$ν=Θ(W-W_{min})$上面[1]式转化为diagonal form：
$$f(ν)\approx f(0)+\frac{1}{2}ν^{T}Λv$$
对应的迭代方式转换为：
$$ν^{[t+1]} =(I-\etaΛ)*ν^{[t]}$$
t+1步相对t步converge比例为：
$$\frac{ν^{[t+1]}}{ν^{[t]}} =\frac{W^{[t+1]}-W_{min}}{W^{[t]}-W_{min}}=(I-\etaΛ)$$
在$w_{i}$维度上缩小幅度为：$$1-η*{λ_{i}}$$
(1) 如果只能用相同的learning rate：$\eta$为scalar \
 · converge的条件：$|1-η*{λ_{i}}| < 1$ ⇒ $\eta<\frac{2}{\underset{i}{max}{λ_{i}}}$ \
 · 在不发生diverge的条件下，实现最快收敛的条件：$\eta_{opt}=\frac{1}{\underset{i}{max}{λ_{i}}}$ \
(2) 如果不同维度用不同的learning rate：取$η$为vector，$\eta=Λ^{-1}$

#### II.2.2 Batch GD收敛速度
1. 收敛速度分析
    - condition number：$κ:=\frac{\underset{i}{max}{λ_{i}}}{\underset{i}{min}{λ_{i}}}$
    - 如果取$η=η_{opt}=\frac{1}{\underset{i}{max}{λ_{i}}}$，$\frac{ν^{[t+1]}}{ν^{[t]}}=\frac{W^{[t+1]}-W_{min}}{W^{[t]}-W_{min}}=(I-\frac{1}{\underset{i}{max}{λ_{i}}}Λ)$
    - 每个维度j上的收敛情况：$R_{j}=\frac{w_{j}^{[t+1]}-w_{j,min}}{w_{j}^{[t]}-w_{j,min}}=1-\frac{λ_{j}}{\underset{i}{max}{λ_{i}}}$
    - 收敛最慢的维度：$R_{min}=1-\frac{\underset{i}{min}{λ_{i}}}{\underset{i}{max}{λ_{i}}}=1-\frac{1}{κ}，(0<R_{min}<1)$
        - condition number越大时，$R_{min}$越大，收敛越慢
        - 对各个维度j都有：$\frac{w_{j}^{[t]}-w_{j,min}}{w_{j}^{[0]}-w_{j,min}}=R_{j}^{[t]}*R_{j}^{[t-1]}*···*R_{j}^{[1]}<R_{min}^{t}$
        - t次迭代后，$\frac{W^{[t]}-W_{min}}{W^{[0]}-W_{min}}<R_{min}^{t}$，因此，凸函数以指数速度收敛
        - 对$\forallε>0$，要让$\frac{W^{[t]}-W_{min}}{W^{[0]}-W_{min}}<ε$，只要$R_{min}^{t}<ε$，此时$t\approx\frac{logε}{logR_{min}}$
        - 因此，converge to ε 所需的迭代步数t的复杂度是$O(log(\frac{1}{ε}))$
            - 注：复杂度不是$O(log(ε))$，因为$log(ε)$是负数
            - $t\approx\frac{logε}{logR}=\frac{log(\frac{1}{ε})}{log(\frac{1}{R})}$，将$log(\frac{1}{R})$视为常数C，得到$t\approx \frac{1}{C}*{log\frac{1}{ε}}$
2. **该方法的问题：**
    - 求Hessian和Hessian分解的复杂度都是$O(n^{3})$
    - condition number大的时候收敛慢
    - 无法确保非凸函数的收敛

#### II.2.3 SGD 收敛速度
1. SGD收敛的<font color=green>**充分条件**</font>：<font color=orange>[详见RLnotes的RM章节]</font> \
(1) $\underset{k}{Σ}\ η_{k}=∞$ ：可以在整个w的取值空间中寻址。 k表示迭代的步数 \
(2) $\underset{k}{Σ}\ η_{k}^{2}<∞$：shrink the step
2. 满足上述充分条件的fastest converging series：$\eta_{opt}\propto \frac{1}{k}$
3. 收敛速度: <font color=norange>**converge to ε 所需的迭代步数t的复杂度是$O(\frac{1}{ε})$**</font> \
(1) 假如$\eta=\eta_{opt}$，则k次iteration之后：$|\frac{W^{[t]}-W_{min}}{W^{[0]}-W_{min}}|<\frac{1}{k}$ \
(2) 对$\forallε>0$，要让$\frac{W^{[t]}-W_{min}}{W^{[0]}-W_{min}}<ε$，只要$\frac{1}{k}=ε$，此时$k= \frac{1}{ε}$